In [ ]:
!git clone https://github.com/michaelwwk/appStoreAnalytics.git

Cloning into 'appStoreAnalytics'...
remote: Enumerating objects: 3652, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 3652 (delta 425), reused 685 (delta 411), pack-reused 2950
Receiving objects: 100% (3652/3652), 231.73 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (2398/2398), done.


In [ ]:
!pip install pygsheets
import pygsheets
import pandas as pd
import numpy as np
import json
!pip install pyspark
from pyspark.sql.functions import split, lower
from pyspark.sql.functions import concat, col, lit
from google.cloud import bigquery, storage
from pyspark.sql import SparkSession
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import lit, lower, concat_ws, col, split
from pyspark.ml.feature import Tokenizer
import sklearn.preprocessing as pp
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=8d04cf8e3c684757376efbb11211ebfbe89198f4da29ac87f0f35cbea24e9845
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Start Spark session
spark = SparkSession.builder.master("local").appName("appStoreAnalytics").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
# googleAPI_json_path = "/content/drive/MyDrive/Colab Notebooks/big-data-analytics-415801-f8364cc7c7d5.json"
# project_id = "big-data-analytics-415801"
# client = bigquery.Client.from_service_account_json(googleAPI_json_path, project = project_id)

In [ ]:
# Define your BigQuery project ID and table ID
project_id = "big-data-analytics-415801"
table_id = 'dev_cleanData.cleanGoogleMain'

# SQL query to fetch the data from the table
sql_query = f"SELECT * FROM {table_id} limit 5000"

# Read data from BigQuery into a DataFrame
googleMain = pd.read_gbq(sql_query, project_id=project_id)

In [ ]:
googleMain.head()

,title,description,summary,installs,minInstalls,realInstalls,score,ratings,reviews,price,free,currency,offersIAP,inAppProductPrice,developer,genre,genreId,contentRating,contentRatingDescription,adSupported,released,lastUpdatedOn,version,appId,categories_list,min_inAppProductPrice,max_inAppProductPrice,1starrating_no,2starrating_no,3starrating_no,4starrating_no,5starrating_no
0,Personal Safety,Personal Safety is an app that helps you prepa...,Quickly connect to help in an emergency.,1+,1,1,4.0465117,47914,1648,0.0,True,USD,False,None,Google LLC,Tools,TOOLS,Everyone,None,False,"Sep 30, 2019","Mar 28, 2024",None,com.google.android.apps.safetyhub,[Tools],None,None,7720,1671,3024,3740,31756
1,Anti Block Wiki,A simple application that allows you to enter ...,Allows you to access Wikipedia.,1+,1,1,4.6,5,0,0.0,True,USD,False,None,Eren Sarıgül,Tools,TOOLS,Everyone,None,True,"Nov 13, 2019","Nov 13, 2019",1.0,info.erensarigul.antiblockwiki,[Tools],None,None,0,0,1,0,4
2,Recorder,Recorder brings the power of search to audio r...,"Your audio, now searchable",1+,1,1,3.6395664,11287,1433,0.0,True,USD,False,None,Google LLC,Tools,TOOLS,Everyone,None,False,"Oct 1, 2019","Apr 1, 2024",None,com.google.android.apps.recorder,[Tools],None,None,2599,578,1007,1190,5902
3,Motion Sense Bridge,This enables Motion Sense interactivity for ke...,Enables Motion Sense experiences with select apps,1+,1,1,3.76,948,131,0.0,True,USD,False,None,Google LLC,Tools,TOOLS,Everyone,None,False,"Oct 15, 2019","May 19, 2020",1.0.305791569,com.google.android.apps.motionsense.bridge,[Tools],None,None,188,47,94,84,529
4,Bubble Shooter,Bubble Shooter is a fun and addictive bubble s...,Bubble Shooter is a fun and addictive bubble s...,1+,1,1,4.75,125,4,0.0,True,USD,False,None,Classic Puzzle Games Studio,Puzzle,GAME_PUZZLE,Everyone,None,True,"Oct 23, 2018","Oct 23, 2018",1.0.0,com.classicpuzzlegames.bubbleshooter,[Puzzle],None,None,0,0,10,10,104


## Data preparation

In [ ]:
filter_list = []

def preprocess_text(text):
    # Tokenize the input text
    tokens = word_tokenize(text)

    # Convert all characters to lower case
    tokens = [t.lower() for t in tokens]

    # Remove Stopwords and filtered words
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words and t not in filter_list]

    # Remove Punctuations
    tokens = [t for t in tokens if t not in string.punctuation]

    # Remove Numbers/Numerics
    tokens = [t for t in tokens if not t.isnumeric()]

    # Lemmatize
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return tokens

In [ ]:
text = googleMain['title'] + ' ' + googleMain['description']

In [ ]:
tfidf = TfidfVectorizer(tokenizer = preprocess_text, min_df = 2, max_df = 0.7)
tdm = tfidf.fit_transform(text)

In [ ]:
tdm

<5000x10797 sparse matrix of type '<class 'numpy.float64'>'
	with 225371 stored elements in Compressed Sparse Row format>

In [ ]:
n_components = 50  # This no. must be the same or smaller than the no. of rows!! Typically set between 100 to 300
svd = TruncatedSVD(n_components=n_components)
tfidf_reduced = svd.fit_transform(tdm)
tfidf_df = pd.DataFrame(tfidf_reduced, columns=[f'component_{i}' for i in range(n_components)])

In [ ]:
googleMain.columns

Index(['title', 'description', 'summary', 'installs', 'minInstalls',
       'realInstalls', 'score', 'ratings', 'reviews', 'price', 'free',
       'currency', 'offersIAP', 'inAppProductPrice', 'developer', 'genre',
       'genreId', 'contentRating', 'contentRatingDescription', 'adSupported',
       'released', 'lastUpdatedOn', 'version', 'appId', 'categories_list',
       'min_inAppProductPrice', 'max_inAppProductPrice', '1starrating_no',
       '2starrating_no', '3starrating_no', '4starrating_no', '5starrating_no'],
      dtype='object')

In [ ]:
googleMainFiltered= googleMain[['realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'adSupported']].loc[googleMain.title != 'None']
googleMainFiltered.head()

,realInstalls,score,reviews,price,free,genre,contentRating,adSupported
0,1,4.0465117,1648,0.0,True,Tools,Everyone,False
1,1,4.6,0,0.0,True,Tools,Everyone,True
2,1,3.6395664,1433,0.0,True,Tools,Everyone,False
3,1,3.76,131,0.0,True,Tools,Everyone,False
4,1,4.75,4,0.0,True,Puzzle,Everyone,True


In [ ]:
googleMainFiltered.columns

Index(['realInstalls', 'score', 'reviews', 'price', 'free', 'genre',
       'contentRating', 'adSupported'],
      dtype='object')

In [ ]:
# create dummy variables (one hot)
googleMainFiltered['free'] = googleMainFiltered['free'].map({'False':0, 'True':1}).astype('int64')
googleMainFiltered['adSupported'] = googleMainFiltered['adSupported'].map({'False':0, 'True':1}).astype('int64')

googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['genre'], prefix='genre')], axis=1)
googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['contentRating'], prefix='contentRating')], axis=1)
googleMainFiltered.pop('genre')
googleMainFiltered.pop('contentRating')

for column in googleMainFiltered.columns:
  if column not in ['realInstalls', 'score', 'reviews', 'price', 'free', 'genre',
       'contentRating', 'adSupported']:
       googleMainFiltered[column] = googleMainFiltered[column].astype(int)

In [ ]:
structured_df = googleMainFiltered.copy() # option 1 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds')
unstructured_df = tfidf_df # option 2 ('title', 'description')
mixed_df = pd.concat([googleMainFiltered, tfidf_df], axis = 1) # option 3 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds' + 'title', 'description', 'summary')

df_list = [structured_df, unstructured_df, mixed_df]

for df in df_list:
  try:
    df.replace('None', np.nan, inplace=True)
  except:
    pass
  try:
    df.replace('nan', np.nan, inplace=True)
  except:
    pass
  try:
    df = df.dropna()
  except:
    pass
  try:
    df = df.astype('float64')
  except:
    pass

In [ ]:
structured_df.head()

,realInstalls,score,reviews,price,free,adSupported,genre_Action,genre_Adventure,genre_Arcade,genre_Art & Design,genre_Auto & Vehicles,genre_Beauty,genre_Board,genre_Books & Reference,genre_Business,genre_Card,genre_Casino,genre_Casual,genre_Comics,genre_Communication,genre_Dating,genre_Education,genre_Educational,genre_Entertainment,genre_Events,genre_Finance,genre_Food & Drink,genre_Health & Fitness,genre_House & Home,genre_Libraries & Demo,genre_Lifestyle,genre_Maps & Navigation,genre_Medical,genre_Music,genre_Music & Audio,genre_News & Magazines,genre_Personalization,genre_Photography,genre_Productivity,genre_Puzzle,genre_Racing,genre_Role Playing,genre_Shopping,genre_Simulation,genre_Social,genre_Sports,genre_Strategy,genre_Tools,genre_Travel & Local,genre_Trivia,genre_Video Players & Editors,genre_Weather,genre_Word,contentRating_Everyone,contentRating_Everyone 10+,contentRating_Mature 17+,contentRating_Teen,contentRating_Unrated
0,1,4.0465117,1648,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,1,4.6,0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,1,3.6395664,1433,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,1,3.76,131,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,1,4.75,4,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
for df in [structured_df, unstructured_df, mixed_df]:
  print(df.shape)

(5000, 58)
(5000, 50)
(5000, 108)


In [ ]:
# import pandas as pd
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# while True:
#     vif = pd.DataFrame()
#     vif["Features"] = googleMainFiltered.columns
#     vif["VIF"] = [variance_inflation_factor(googleMainFiltered.values, i) for i in range(googleMainFiltered.shape[1])]

#     max_vif = vif["VIF"].max()

#     if max_vif > 10:
#         column_to_remove = vif.loc[vif["VIF"].idxmax(), "Features"]
#         googleMainFiltered = googleMainFiltered.drop(column_to_remove, axis=1)
#     else:
#         break

#### K-Nearest Neighbours

In [ ]:
# <MODEL PREP>

def knn_showResults(qind, dis, ind, df_condensed, df):
  for i, d in zip(ind[qind], dis[qind]) :
    #retrieve the original row index, to access 'data'
    reali = df_condensed.loc[i, 'index']
    print('=====')
    print('\tRow Index: ' + str(reali) + '\tDistance: ' + str(d) + '\tApp Title: ' + df.loc[reali, 'title'] )
    #access the values in 'df_condensed' using new row index
    # print(str(df_condensed.loc[i]))
    print(df.loc[reali, 'title'] + "\t"+ df.loc[reali, 'description'])

#### Cosine-Similarity

In [ ]:
# <MODEL PREP>

def cosine_showResults(q, k, df, tdm):
    # print(df['title'][q])
    vals = cosine_similarity(tdm[q], tdm)
    idx_asc = vals.argsort()[0][-k:] #get index of highest sim values
    idx_dsc = idx_asc[::-1] #reverse the indices
    # print(idx_dsc)
    flatv = np.sort(vals[0])
    vk_asc = flatv[-k:]
    vk_dsc = vk_asc[::-1] #reverse the list
    if(vk_dsc[1]==0):
        print("No similar app is found")
    else:
        for v, i in zip(vk_dsc, idx_dsc):
          print('=====')
          print(df['title'][i]+'\t'+str(v))
          print('\t'+df['description'][i])

#### Embedding method (Doc2Vec model)

In [ ]:
# <MODEL PREP>

googleMain['textonly'] = googleMain['title'] + ' ' + googleMain['description']
textonly = googleMain['textonly'].fillna('')
text_tokens = [word_tokenize(t.lower()) for t in textonly]
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
model = Doc2Vec(vector_size=64, min_count=2, epochs=40)
model.build_vocab(tagged_data)
print("Data loaded into model.")

# Train the model using our data
print("Training model ..")
model.train(tagged_data, total_examples=model.corpus_count, epochs=40)
print("Model trained.")

Data loaded into model.
Training model ..
Model trained.


In [ ]:
# googleMain.to_csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

# # Read CSV files into PySpark DataFrame
# googleMain_sparkDF = spark.read.format("csv") \
#                     .option("inferSchema", "true") \
#                     .option("header", "true") \
#                     .option("multiline", "true") \
#                     .option("escape", "\"") \
#                     .csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

In [ ]:
# # Create "text" column by concatenating title, description, and summary
# googleMain_sparkDF = googleMain_sparkDF.withColumn('text', concat(col('title'), lit(' '), col('description')))

# # Tokenize the text and store it in the "text_tokens" column
# googleMain_sparkDF = googleMain_sparkDF.withColumn("text_tokens", split(lower("text"), "\s+"))

# # Select only the "text_tokens" column and collect it into a list
# text_tokens = googleMain_sparkDF.select("text_tokens").rdd.flatMap(lambda x: x).collect()

In [ ]:
# # Load data into model
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
# model = Doc2Vec(vector_size=64, min_count=2, epochs=40)
# model.build_vocab(tagged_data)

# # Train the model using our data
# model.train(tagged_data, total_examples=model.corpus_count, epochs=40)

In [ ]:
model.save("/content/drive/MyDrive/googleRecModel.model")

## Run model results

#### K-Nearest Neighbours

In [ ]:
try:
  googleMain.reset_index(inplace=True)
except:
  pass
googleMainFiltered.reset_index(inplace=True)

In [ ]:
"""
<INFERENCE>

Per sample, the distance scores are generally closer to 0 (which is better) than structured_df & mixed_df.
Also, the unstructured_df outputs have more accurate results intuitively.
"""

appIndex_list = [45,20,15]

for sample_index, appIndex in enumerate(appIndex_list):
  for index, df in enumerate(df_list):

    scaler = pp.MinMaxScaler()
    googleMainFilteredScaled = scaler.fit_transform(df)

    nbrs_scaled = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(googleMainFilteredScaled)
    dis_scaled, ind_scaled = nbrs_scaled.kneighbors(googleMainFilteredScaled)

    print(f"Sample {sample_index+1}:\n")
    if index == 0:
      print('<<structured_df>>') # [structured_df, unstructured_df, mixed_df]
    if index == 1:
      print('<<unstructured_df>>') # [structured_df, unstructured_df, mixed_df]
    if index == 2:
      print('<<mixed_df>>') # [structured_df, unstructured_df, mixed_df]
    knn_showResults(appIndex, dis_scaled, ind_scaled, googleMainFiltered, googleMain)
    print('\n')

Sample 1:

<<structured_df>>
=====
	Row Index: 45	Distance: 0.0	App Title: Password Guardian
Password Guardian	PasssGuard
=====
	Row Index: 100	Distance: 1.4142150654157255	App Title: Fun! CPU!
Fun! CPU!	Image shows CPU usage at any time
=====
	Row Index: 113	Distance: 1.4142376108634889	App Title: Astral Void - Sci-Fi Game
Astral Void - Sci-Fi Game	2D Sci-Fi Space Game - Play as a rocket that saves astronauts
=====
	Row Index: 161	Distance: 1.4142511379596299	App Title: Slappy Paddles
Slappy Paddles	Test your reflexes in this single player reimaging of the classic arcade game.
=====
	Row Index: 119	Distance: 1.4143097538811917	App Title: The Hera Hera Kingdom DX
The Hera Hera Kingdom DX	Retro atmosphere
With 2D action puzzle game
3D shooting game
It became one application!
=====
	Row Index: 202	Distance: 1.4143097538811917	App Title: SHUFFLE 2D Arcade
SHUFFLE 2D Arcade	Endless Shoot 'em up with an arcade base


Sample 1:

<<unstructured_df>>
=====
	Row Index: 45	Distance: 0.0	App Titl

#### Cosine-Similarity

In [ ]:
"""
<INFERENCE>

Since distance and similarity score are not equal, we cant use the scoring to compare "apple to apple".
However, from the intuitive interpretation of results, more results per sample for this algo are likely to contain key words (password; card, game; wallpapers) than using KNN.
For sample 3, more wallpapers are of "nature" origin than KNN's (although both algo have almost all results as wallpapers).
"""

for sample_index, appIndex in enumerate(appIndex_list):

  print(f"Sample {sample_index+1}:\n")
  cosine_showResults(appIndex, 6, googleMain, tdm) # tdm only
  print("\n")

Sample 1:

=====
Password Guardian	1.0000000000000002
	PasssGuard
=====
Security App	0.31877586722094053
	All your passwords in one place! "Your safety is our duty!"
=====
Roundshield password generator	0.31808545090310913
	Password app with three levels of security: Token, Generated and Saved Passwords
=====
Unmask - Your Privacy	0.30885446531275673
	Unmask allows you to securely store and access all your passwords in one place.
=====
Masterkey	0.2834863116198093
	Masterkey is an easy to use password manager for the Android platform.
=====
NEKTA CLOUD	0.2720577829239205
	NEKTA CLOUD is a unique service for collecting and processing data in IoT.


Sample 2:

=====
Assortments	1.0
	A matching game in which three cards are chosen based on their characteristics.
=====
Нока - игра в карты	0.334084562027729
	Noka is an interesting new card game in which cheating is ruled out.
=====
XV Solitaire	0.30631432309873796
	The elemental card game of fifteens.
=====
Square Scarf (Kerchief)	0.3053386

#### Embedding method (Doc2Vec model)

In [ ]:
appId_list = googleMain[googleMain.index.isin(appIndex_list)].reset_index(drop = True).appId.to_list()

['com.wallpaper.media.dev.tyas.naturebeautifulwallpaperwallpaperalam',
 'com.jackscompany.set',
 'io.ionic.password']

In [ ]:
# googleMain.to_csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

# # Read CSV files into PySpark DataFrame
# googleMain_sparkDF = spark.read.format("csv") \
#                     .option("inferSchema", "true") \
#                     .option("header", "true") \
#                     .option("multiline", "true") \
#                     .option("escape", "\"") \
#                     .csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

In [ ]:
# googleMain_sparkDF_test = googleMain_sparkDF.where(col("appId").isin(appId_list))
# googleMain_sparkDF_test.show()

In [ ]:
# # Concatenate the columns into a single column "text"
# googleMain_sparkDF_test = googleMain_sparkDF_test.withColumn(
#     "text",
#     concat_ws(" ",
#             lower(col('title')),
#             lower(col(f"`{'description'}`"))
#             )
# )

# # # Tokenize the text column
# # tokenizer = Tokenizer(inputCol="text", outputCol="text_tokens")
# # googleMain_sparkDF_test = tokenizer.transform(googleMain_sparkDF_test)

# # Iterate over each row and compute similarity scores
# index = 0
# for row in googleMain_sparkDF_test.collect():
#   index += 1
#   test_vec = model.infer_vector(row["text_tokens"])
#   results = model.docvecs.most_similar(positive=[test_vec], topn=5)

#   #check the results. Do they make sense?
#   for i, (doc_id, similarity_score) in enumerate(results):
#       title = googleMain_sparkDF.select("title").collect()[doc_id][0]
#       text = googleMain_sparkDF.select("summary").collect()[doc_id][0]
#       print(f"\nSample {index}:\n")
#       print(f"Score:\n{similarity_score}\n")
#       print(f"Title:\n{title}\n")
#       print(f"Details:\n{text}\n")
#       print("=====" )

In [ ]:
# # Define your BigQuery project ID and table ID
# project_id = "big-data-analytics-415801"
# table_id = 'dev_cleanData.cleanGoogleMain'

# # SQL query to fetch the data from the table
# sql_query = f"SELECT * FROM {table_id} limit 50000"

# # Read data from BigQuery into a DataFrame
# googleMain = pd.read_gbq(sql_query, project_id=project_id)

In [ ]:
# googleMain['textonly'] = googleMain['title'] + ' ' + googleMain['description']
# textonly = googleMain['textonly'].fillna('')
# text_tokens = [word_tokenize(t.lower()) for t in textonly]
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
# model = Doc2Vec(vector_size=64, min_count=2, epochs=40)
# model.build_vocab(tagged_data)
# print("Data loaded into model.")

# # Train the model using our data
# print("Training model ..")
# model.train(tagged_data, total_examples=model.corpus_count, epochs=40)
# print("Model trained.")

In [ ]:
# googleMain_reduced = googleMain[googleMain.index.isin(appIndex_list)]
# googleMain_reduced

In [ ]:
"""
<INFERENCE>

Similarity scores are better when the text corpus are represented by embeddings than TDM.
"""

googleMain_reduced = googleMain[googleMain.appId == appId_list[-1]]

sample_index = 0
# Iterate over each row and compute similarity scores
for index, series in googleMain_reduced.iterrows():

  sample_index += 1
  newData = series['title'] + ' ' + series['description']

  test_doc = word_tokenize(newData.lower())
  test_vec = model.infer_vector(test_doc)
  results = model.docvecs.most_similar(positive=[test_vec], topn=6)

  for i, (index, similarity_score) in enumerate(results):

      title = googleMain["title"][index]
      description = googleMain["summary"][index]
      # print(f"Sample {sample_index}:\n")
      # print(f"[Result {i+1}]\n")
      print(f"App Name:\n{title}\n")
      print(f"Description:\n{description}\n")
      print(f"Score:\n{similarity_score}")
      print("=====" )

App Name:
Password Guardian

Description:
PasssGuard

Score:
0.9467177987098694
=====
App Name:
kpnPass Password Manager

Description:
kpnPass Password Manager is easy password manager

Score:
0.6490378975868225
=====
App Name:
Password Cerber

Description:
Intuitive application for storing passwords on your phone in a safe way.

Score:
0.642838716506958
=====
App Name:
KG Password Generator - Genera

Description:
Generates passwords that are unique. Provides a SuperGenPass generator as well.

Score:
0.5938439965248108
=====
App Name:
MultiPass

Description:
Secure your identification and access codes!

Score:
0.5927690863609314
=====
App Name:
TriGen

Description:
64G quantum random number secured USB Flash Drive

Score:
0.5873583555221558
=====


## IGNORE THIS PART!
### If want to reference codes for reading Google forms 'synthetic apps', refer to model.py on GitHub main branch.

## Chosen method:
#### Embedding method (Doc2Vec model)

In [ ]:
# Define the scope of the Google Sheets API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Set the path to the service account key JSON file
service_account_key_path = '/content/drive/MyDrive/big-data-analytics-415801-f8364cc7c7d5.json'

# Authenticate with Google Sheets API using the service account key
# credentials = ServiceAccountCredentials.from_json_keyfile_name(service_account_key_path, scope)
gspread_client = pygsheets.authorize(service_file = service_account_key_path)

# Open the Google Sheets by its URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1zo96WvtgcfznAmSjlQJpnbKIX_NfSIMpsdLcrJOYctw/edit#gid=0'
spreadsheet = gspread_client.open_by_url(sheet_url)

# Select the specific worksheet by its title or index
worksheet = spreadsheet.sheet1  # Change to the desired sheet

# Read data from the worksheet
data = worksheet.get_all_records()
# newApplications_df = pd.DataFrame(data)
newApplications_df = spark.createDataFrame(data)

newApplications_df = newApplications_df.filter(
    (newApplications_df["Will you be publishing your application to Apple App Store, Google Play Store, or both?"] == "Google Play Store") |
    (newApplications_df["Will you be publishing your application to Apple App Store, Google Play Store, or both?"] == "Both")
)

newApplications_df.show()

+--------------------------------------------------+----------------------------------------------------+-----------------+-----------------------------------------+---------------------------------------------------------------------------------------+
|Please provide a description for your application.|Please provide a short summary for your application.|        Timestamp|What is the name of your new application?|Will you be publishing your application to Apple App Store, Google Play Store, or both?|
+--------------------------------------------------+----------------------------------------------------+-----------------+-----------------------------------------+---------------------------------------------------------------------------------------+
|                                               bbb|                                                 ccc|4/8/2024 15:58:44|                                      aaa|                                                                      Goo

In [ ]:
# Define the schema
schema = StructType([
    StructField("newApp", StringType(), nullable=True),
    StructField("appRank", StringType(), nullable=True),
    StructField("appName", StringType(), nullable=True),
    StructField("appId", StringType(), nullable=True),
    StructField("appScore", DoubleType(), nullable=True)
])

# Create an empty DataFrame with the specified schema
df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

# Concatenate the columns into a single column "text"
newApplications_df = newApplications_df.withColumn(
    "text",
    concat_ws(" ",
              lower(col("What is the name of your new application?")),
              lower(col("`Please provide a description for your application.`")),
              lower(col("`Please provide a short summary for your application.`")))
)

# Tokenize the text column
tokenizer = Tokenizer(inputCol="text", outputCol="text_tokens")
newApplications_df = tokenizer.transform(newApplications_df)

In [ ]:
# Iterate over each row and compute similarity scores
for row in newApplications_df.collect():
    test_vec = model.infer_vector(row["text_tokens"])
    results = model.docvecs.most_similar(positive=[test_vec], topn=5)
    print("[Original]\n")
    print(f"Title: {row['What is the name of your new application?']}")
    print(f"Description: {row['Please provide a description for your application.']}")
    print(f"Summary: {row['Please provide a short summary for your application.']}")
    print("-" * 50)
    # Iterate over the results and append rows to the DataFrame
    for i, (doc_id, similarity_score) in enumerate(results):
        title = googleMain_sparkDF.select("title").collect()[doc_id][0]
        id = googleMain_sparkDF.select("appId").collect()[doc_id][0]
        text = googleMain_sparkDF.select("text").collect()[doc_id][0]
        print(f"[Result {i+1}]\n")
        print(f"Score:\n{similarity_score}\n")
        print(f"Title:\n{title}\n")
        print(f"Details:\n{text}\n")
        print("-" * 50)

        # Create a new row
        new_row = (row["What is the name of your new application?"], str(i+1), title, id, similarity_score)

        # Append the row to the DataFrame
        df = df.union(spark.createDataFrame([new_row], schema=schema))

# Filter out the empty row
df = df.filter(df.newApp.isNotNull())

[Original]

Title: aaa
Description: bbb
Summary: ccc
--------------------------------------------------
[Result 1]

Score:
0.8377600908279419

Title:
のこのこキノコ

Details:
のこのこキノコ ●遊び方―――――――――――――――――――――――――――――――――――――――――

①同じキノコを3つ以上くっつけて新しいキノコを作ろう
　・キノコのランクが1つあがるよ

②机に霧吹きで水を与えて湿らせよう
　・キノコがすぐ生えるよ

③キノコから発生したエネルギーを使って新しく出現する仲間を解放しよう
　・仲間を解放すると、そのランクのキノコが直接机から生えてくるよ

④霧吹きのお水がなくなったら、外で水滴を集めよう　
　・水滴を集めるとまた霧吹きが使えるようになるよ

⑤フィーバータイムは窓から大量の霧が入ってきてたくさんのキノコが生えてくるよ
　・たくさんくっつけてコンボを狙おう
　
●仲間のキノコ――――――――――――――――――――――――――――――――――――――

1	まるまるキノコ・・・【机に生えたキノコの生き残り。大切に育てよう。】
2	しいたけ・・・【うまみが詰まっているキノコ。とてもおいしい。】
3	しましまキノコ・・・【しましまのキノコ。一応、食べても大丈夫。】
4	えのき・・・【ほっそりとしたキノコ。鍋に入っていることが多い。】
5	エレキノコ・・・【電気をたくわえている。毒キノコではないらしい。】
6	エリンギ・・・【大型のキノコ。焼いて食べてみよう。】
7	ごつごつキノコ・・・【岩のように硬い。人に向かって投げると危険。】
8	まいたけ・・・【かさが特徴的なキノコ。おどりだすくらいおいしい？】
9	カチコチキノコ・・・【常に凍っているキノコ。無理に解凍すると溶けてしまう。】
10	なめこ・・・【つるつるプルプルしている。家庭栽培も可能なキノコ。】
11	モクモクキノコ・・・【木目模様のキノコ。意外とやわらかい。】
12	マッシュルーム・・・【洋食料理によく使われる。成熟すると、かさの裏が黒くなる。】
13	メタルキノコ・・・【すごく重く、少量でも満腹になる。ダイエットに効果が

In [ ]:
df.show()

+-------+-------+------------------------------------+--------------------+------------------+
| newApp|appRank|                             appName|               appId|          appScore|
+-------+-------+------------------------------------+--------------------+------------------+
|    aaa|      1|                      のこのこキノコ|air.jp.globalgear...|0.8377600908279419|
|    aaa|      2|脱出ゲーム ストレンジミュージ...|com.nankaiigame.misu|0.8244614601135254|
|    aaa|      3|  大和証券　マイナンバー登録用アプリ|jp.co.dir.ss.moe....|0.8220852613449097|
|    aaa|      4|                Maharashtra Medic...|com.maharashtrame...|0.8091552257537842|
|    aaa|      5|                Technosoft techno...|com.fst.apps.tech...|0.8080146312713623|
|      q|      1|                      のこのこキノコ|air.jp.globalgear...|0.9380485415458679|
|      q|      2|                                 BGI|       co.nick.tmokb|0.9047492146492004|
|      q|      3|  大和証券　マイナンバー登録用アプリ|jp.co.dir.ss.moe....|0.9028522372245789|
|      q|      4|  

In [ ]:
# # Prep data to import into GBQ

# df = pd.DataFrame(columns = ["newApp", "appRank", "appName", "appId", "appScore"])

# for index, series in newApplications_df.iterrows():

#   newData = series['What is the name of your new application?'] + ' ' + \
#   series['Please provide a description for your application.'] + ' ' + \
#   series['Please provide a short summary for your application.']

#   test_doc = word_tokenize(newData.lower())
#   test_vec = model.infer_vector(test_doc)
#   results = model.docvecs.most_similar(positive=[test_vec], topn=5)

#   #check the results. Do they make sense?
#   for i, (doc_id, similarity_score) in enumerate(results):
#       title = googleMain_sparkDF.select("title").collect()[doc_id][0]
#       id = googleMain_sparkDF.select("appId").collect()[doc_id][0]
#       text = googleMain_sparkDF.select("text").collect()[doc_id][0]

#       row = [series['What is the name of your new application?'], i+1, title, id, similarity_score]
#       df.loc[len(df)] = row

# df

In [ ]:
# Stop Spark session
spark.stop()